In [1]:
from nansat import Nansat, Domain
import matplotlib.pyplot as plt
import numpy as np
from boreali import  Boreali, lm
import custrepr
from matplotlib.patches import Rectangle

/home/artemm/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# different kinds of test data was processed by boreali algorithms 
# october data
n_osw_oct = Nansat('/home/artemm/Documents/michigan/tests/A2014301181500.L2_LAC_OC.x.nccpa_OSW.nc')
n_deep_oct = Nansat('/home/artemm/Documents/michigan/tests/A2014301181500.L2_LAC_OC.x.nccpa_deep.nc')
# may data
n_osw_may = Nansat('/home/artemm/Documents/michigan/tests/A2014121184000.L2_LAC_OC.x.nccpa_OSW.nc')
n_deep_may = Nansat('/home/artemm/Documents/michigan/tests/A2014121184000.L2_LAC_OC.x.nccpa_deep.nc')

# base files
n_rrs_oct = Nansat('/home/artemm/Documents/michigan/data/2014/oct/A2014301181500.L2_LAC_OC.x.nc')
n_rrs_may = Nansat('/home/artemm/Documents/michigan/data/2014/may/A2014121184000.L2_LAC_OC.x.nc')

# bathymetry
bottom = Nansat('/home/artemm/Documents/michigan/tests/michigan_lld.grd')

dom = Domain('+proj=latlong +datum=WGS84 +ellps=WGS84 +no_defs', '-lle -86.3 44.6 -85.2 45.3 -ts 300 200')

n_rrs_oct.reproject(dom)
n_rrs_may.reproject(dom)
bottom.reproject(dom)

=>michigan.grd<=


In [10]:
# droving of bottom
h = np.copy(bottom[1])
h[np.where(h > np.float32(0.0))] = np.nan
h[np.where(h < np.float32(0.0))] *= np.float32(-1)
plt.figure('bottom')
plt.text(50, 102, '1', fontsize=15, )

plt.imshow(h)
plt.colorbar()

# adding of rectangles for each region
currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((112 - .5, 0 - .5), 17, 20, fill=False))    # bank near S.Fox isl.
currentAxis.add_patch(Rectangle((14 - .5, 86 - .5), 46, 48, fill=False))    # Maniton Passage
currentAxis.add_patch(Rectangle((196 - .5, 14 - .5), 68, 38, fill=False))   # Grand Traverse Bay

plt.show()

In [51]:
# droving of Rrsw spectrum for the deep and shallow region 
num_bands = range(2, 12)
n = n_rrs_oct
test_deep = [n[band][9][102] for band in range(2, 12)]
h_deep = bottom[1][9][102]
h_shallow = bottom[1][6][122]
test_shallow = [n[band][6][122] for band in range(2, 12)]
wavelens = [412, 443, 469, 488, 531, 547, 555, 645, 667, 678]
custom_n = Nansat(domain=n)

# Rrsw processing
band_rrs_numbers = list(map(lambda x: n._get_band_number('Rrs_' + str(x)), wavelens))

for index in range(0, len(wavelens)):
    rrsw = n[band_rrs_numbers[index]] / (0.52 + 1.7 * n[band_rrs_numbers[index]])
    custom_n.add_band(rrsw, parameters={'name': 'Rrsw_' + str(wavelens[index]),
                                        'units': 'sr-1',
                                        'wavelength': wavelens[index]})

# plotting
plt.plot(num_bands, test_deep, label='deep ' + str(round(h_deep * -1, 1)) + 'm')
plt.plot(num_bands, test_shallow, label='shallow ' + str(round(h_shallow * -1, 1)) + 'm')
plt.legend()
plt.xticks(num_bands, wavelens)
plt.xlabel('wavelength, nm')
plt.ylabel('Rrsw, sr^-1')
#plt.ylim([0, 0.04])
plt.show()

In [53]:
plt.imshow(n_osw_oct['chl'])
plt.colorbar()
plt.show()

In [1]:
# coding: utf-8
def boreali_osw_processing(final_path):
    """
    Мой код в данной функции основан на tutorial.py который я нашел в репозитории boreali.
    :param obj: путь до изображения
    :param final_path: Путь для сохранения файлов
    :return:
    """
    '''
    Данные для тестирвоания
    obj = '/home/artemm/Documents/michigan/average_data/2014/oct/A2014295301.L2_LAC_OC.x.nc.reproject.nc.mosaic.nc'
    path = '/home/artemm/Documents/work/'
    '''
    wavelen = [412, 443, 469, 488,
               531, 547, 555,
               645, 667, 678]

    cpa_limits = [0.01, 5,
                  0.01, 1,
                  0.01, 1, 10]

    h = custrepr.get_deph()  # Глубина исследуемого района по батиметрии

    b = Boreali('michigan', wavelen)
    n = n_rrs_oct
    dom = Domain('+proj=latlong +datum=WGS84 +ellps=WGS84 +no_defs', '-lle -86.3 44.6 -85.2 45.3 -ts 300 200')
    n.reproject(dom)
    theta = np.zeros_like(n[2])

    custom_n = Nansat(domain=n)
    band_rrs_numbers = list(map(lambda x: n._get_band_number('Rrs_' + str(x)),
                                wavelen))   # Получаем список номеров бандов в которых лежат значения Rrs

    # osw считает по значниям Rrs ?
    # для корректной работы складываем в custom_n значения и Rrs и Rrsw
    for index in range(0, len(wavelen)):
        rrsw = n[band_rrs_numbers[index]] / (0.52 + 1.7 * n[band_rrs_numbers[index]])   # Пересчитываем Rrs в Rrsw
        custom_n.add_band(rrsw, parameters={'name': 'Rrsw_' + str(wavelen[index]),  # Складываем в новый объект Rrsw
                                            'units': 'sr-1',
                                            'wavelength': wavelen[index]})
        # Складываем в новый объект значения Rrs
        custom_n.add_band(n[band_rrs_numbers[index]], parameters={'name': 'Rrs_' + str(wavelen[index]),
                                                                  'units': 'sr-1',
                                                                  'wavelength': wavelen[index]})

    custom_n = custrepr.create_mask(custom_n)
    cpa = b.process(custom_n, cpa_limits,  mask=custom_n['mask'], depth=h, theta=theta, threads=4)

    custom_n.add_band(array=cpa[0], parameters={'name': 'chl',
                                                'long_name': 'Chlorophyl-a',
                                                'units': 'mg m-3'})
    custom_n.add_band(array=cpa[1], parameters={'name': 'tsm',
                                                'long_name': 'Total suspended matter',
                                                'units': 'g m-3'})
    custom_n.add_band(array=cpa[2], parameters={'name': 'doc',
                                                'long_name': 'Dissolved organic carbon',
                                                'units': 'gC m-3'})
    custom_n.add_band(array=cpa[3], parameters={'name': 'mse',
                                                'long_name': 'Root Mean Square Error',
                                                'units': 'sr-1'})
    custom_n.add_band(array=cpa[4], parameters={'name': 'mask',
                                                'long_name': 'L2 Boreali mask',
                                                'units': '1'})

    custom_n.export('cpa_OSW.nc')

    fig_params = {'legend': True,
                  'LEGEND_HEIGHT': 0.5,
                  'NAME_LOCATION_Y': 0,
                  'mask_array': cpa[4],
                  'mask_lut': {1: [255, 255, 255], 2: [128, 128, 128], 4: [200, 200, 255]}}
    custom_n.write_figure(final_path + 'chl_OSW.png', 'chl', clim=[0, 1.], **fig_params)
    custom_n.write_figure(final_path + 'tsm_OSW.png', 'tsm', clim=[0, 1.], **fig_params)
    custom_n.write_figure(final_path + 'doc_OSW.png', 'doc', clim=[0, .2], **fig_params)
    custom_n.write_figure(final_path + 'mse_OSW.png', 'mse', clim=[1e-5, 1e-2],
                          logarithm=True, **fig_params)
    n.write_figure(final_path + 'rgb_OSW.png',
                   [16, 14, 6],
                   clim=[[0, 0, 0], [0.006, 0.04, 0.024]],
                   mask_array=cpa[4],
                   mask_lut={2: [128, 128, 128]})